# Import Libraries

In [3]:
import cv2
import pandas as pd
import os
import joblib
import time
from datetime import datetime
from deepface import DeepFace
import numpy as np

###### initializes the a variable with the directory path where the face images are stored. It also initializes two empty lists to store face encodings and the names. The face cascade variable is set for using OpenCV pre-trained Haar cascade classifier for detecting faces

In [4]:
face_direct = 'ImagesAttendance'  
encodings = []
names= []
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [5]:
#Initialize attendance CSV file
date_folder = datetime.now().strftime("%Y-%m-%d")
os.makedirs(date_folder, exist_ok=True)

#Define the path to the CSV file and create it if it doesn't exist
csv_file = os.path.join(date_folder, 'Attendance.csv')

if not os.path.exists(csv_file):
    pd.DataFrame(columns=['Name', 'Time', 'Image Path']).to_csv(csv_file, index=False)
else:
    print(f"File '{csv_file}' already exists.")

##### THIs initializes the process for managing  attendance CSV file. It starts by creating a directory named with the current date. If this directory does not already exist ensures it exists. Then the path for the CSV file is defined. If the file does not exist, it creates a new CSV file with columns for name,time, and image path. If the file already exists it prints a message this avoids unnecessary file creation.

In [ ]:
def process_image(image_path, person_name):
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Failed to read {image_path}")
        return None
    else:
        try:
            result = DeepFace.represent(img, model_name='Facenet', enforce_detection=False)[0]
            encoding = result['embedding']
            return encoding, person_name
        except Exception as e:
            print(f"Could not process {image_path}: {e}")
            return None

def process_folder(person_folder_path, person_name):
    image_files = [f for f in os.listdir(person_folder_path) if f.endswith(('.jpg', '.jpeg'))]
    
    results = []
    for image_file in image_files:
        image_path = os.path.join(person_folder_path, image_file)
        result = process_image(image_path, person_name)
        if result is not None:
            results.append(result)
    
    return results

def process_directory(face_directory):
    person_folders = [f for f in os.listdir(face_directory) if os.path.isdir(os.path.join(face_directory, f))]
    
    all_results = []
    for folder in person_folders:
        folder_path = os.path.join(face_directory, folder)
        results = process_folder(folder_path, folder)
        if results:
            all_results.extend(results)
    
    return all_results

encodings_names = process_directory(face_direct)

#append the results to the lists
if encodings_names:
    for encoding, name in encodings_names:
        encodings.append(encoding)
        names.append(name)

##### firrst recording the current time and printing a message with the name, time, and the image path. It will then check if the CSV file specified by name exists. If file exists it reads the existing data into a df.creates a new DataFrame entry with the provided name, time, and image path, appends this entry. It then prints a message that tells the attendance has been logged.

In [ ]:
def log_attendance(name, img_path):
    time_now = datetime.now().strftime("%H:%M:%S")
    print(f"Logging attendance: Name: {name}, Time: {time_now}, Image Path: {img_path}")
    
    #Check if the CSV file exists before reading it
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
    else:
        print(f"CSV file {csv_file} does not exist. Creating a new file.")
        df = pd.DataFrame(columns=['Name', 'Time', 'Image Path'])

    #Create a new entry
    new_entry = pd.DataFrame([{'Name': name, 'Time': time_now, 'Image Path': img_path}])
    
    #Concatenate the new entry to the existing DataFrame
    df = pd.concat([df, new_entry], ignore_index=True)
    
    #Write the updated DataFrame back to the CSV file
    df.to_csv(csv_file, index=False)
    print(f"Attendance logged for {name}")

##### A trained age prediction model to estimate the age of faces in images. this loades the model preprocessing images to match the requirements, and making predictions based the images

In [ ]:
#Load trained age prediction model
age_regression_model = joblib.load('MLAssignment2.pk1')  # Replace with your model's path

def preprocess_for_model(image):
    #Replace these steps with the exact preprocessing steps used during your model training
    image_resized = cv2.resize(image, (224, 224))
    #Normalize pixel values
    image_normalized = image_resized / 255.0  
    #Reshape for model input
    image_reshaped = image_normalized.reshape(1, 224, 224, 3)  
    return image_reshaped

def predict_age(face_image):
    if face_image.shape[-1] == 3: 
        face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    else:
        face_image_rgb = face_image
    #Preprocess the image for model
    preprocessed_image = preprocess_for_model(face_image_rgb)
    
    #Predict the age using trained  model
    if preprocessed_image is not None:
        predicted_age = age_regression_model.predict(preprocessed_image)
        return str(int(predicted_age[0]))  # Convert to integer and then to string
    else:
        print("Preprocessing failed.")
        return "N/A"

##### this sets up a real-time face detection and attendance logging system. add new attendance entries to a CSV file. WHich creates a new file if it does not exist and appending data to it directly. then captures video frames from the webcam and processes them to detect faces. If no faces are detected it prints a message and for every detected face it extracts the face region  and then converts it into RGB format which then computes the face encoding. It is then compared with this encoding against the known face encodings to identify who the person is. If the face is unknown it predicts the person's age using a pre-trained model.

In [ ]:
last_log_time = time.time()
log_interval = 3
date_folder = datetime.now().strftime("%Y-%m-%d")
if not os.path.exists(date_folder):
    os.makedirs(date_folder)

def log_attendances(name, img_path):
    time_now = datetime.now().strftime("%H:%M:%S")
    print(f"Logging attendance: Name: {name}, Time: {time_now}, Image Path: {img_path}")

    #Create a DataFrame for the new entry
    new_entry = pd.DataFrame([{'Name': name, 'Time': time_now, 'Image Path': img_path}])

    #Append the new entry to the CSV file directly
    new_entry.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)
    print(f"Attendance logged for {name}")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    print(f"Number of faces detected: {len(faces)}")

    if not faces:
        print("No faces detected")

    for (x, y, w, h) in faces:
        face_image = frame[y:y+h, x:x+w]
        name = "Unknown"
        age = "N/A"

        face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        face_encoding = DeepFace.represent(face_image_rgb, model_name='Facenet', enforce_detection=False)[0]['embedding']

        #check for known faces
        for known_encoding, known_name in zip(known_encodings, known_names):
            result = DeepFace.verify(face_encoding, known_encoding, model_name='Facenet', enforce_detection=False)
            if result['verified']:
                name = known_name
                break

        if name == "Unknown":
            age = predict_age(face_image_rgb)  

        #draw bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        #display name and age above the bounding box
        label = f"{name}, Age: {age}"
        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
        label_w, label_h = label_size
        y_label = max(y - 10, 10)  
        cv2.rectangle(frame, (x, y_label - label_h - 10), (x + label_w, y_label), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, label, (x, y_label - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

        #Save image and log attendance
        img_path = os.path.join(date_folder, f"{name}_{datetime.now().strftime('%H%M%S')}.jpg")
        cv2.imwrite(img_path, face_image)

        current_time = time.time()
        if name != "Unknown" and (current_time - last_log_time) > log_interval:
            log_attendance(name, img_path)
            last_log_time = current_time

    cv2.imshow('Real-Time Face Detection and Attendance System', frame)

    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

cap.release()
cv2.destroyAllWindows()